In [3]:
from vortice.config_def import SiteConfig


cachoeira = SiteConfig(
        name="CS",
        lat=-30.446667,
        lon=-53.194444,
        alt=73.0,
        canopy_height=0.1
        )


AttributeError: 'SiteConfig' object has no attribute 'roughness_length'